# Training Item Response Theory (IRT) models

In this notebook, we show how to train your own Item Response Theory (IRT) models.

## Preparing data

Loading packages

In [1]:
import numpy as np
import pickle
from tqdm import tqdm
from irt import *
from utils import *

random_state = 42

The leaderboard dataset we will use is composed by six scenarios (sub-datasets):
1. TruthfulQA
1. GSM8K
1. Winogrande
1. ARC
1. HellaSwag
1. MMLU

MMLU is further divided into sub-scenarios (e.g., abstract algebra, anatomy, etc). Let's check scenarios and sub-scenarios:

In [2]:
scenarios

{'harness_truthfulqa_mc_0': ['harness_truthfulqa_mc_0'],
 'gsm8k': ['harness_gsm8k_5'],
 'winogrande': ['harness_winogrande_5'],
 'arc': ['harness_arc_challenge_25'],
 'hellaswag': ['harness_hellaswag_10'],
 'mmlu': ['harness_hendrycksTest_abstract_algebra_5',
  'harness_hendrycksTest_anatomy_5',
  'harness_hendrycksTest_astronomy_5',
  'harness_hendrycksTest_business_ethics_5',
  'harness_hendrycksTest_clinical_knowledge_5',
  'harness_hendrycksTest_college_biology_5',
  'harness_hendrycksTest_college_chemistry_5',
  'harness_hendrycksTest_college_computer_science_5',
  'harness_hendrycksTest_college_mathematics_5',
  'harness_hendrycksTest_college_medicine_5',
  'harness_hendrycksTest_college_physics_5',
  'harness_hendrycksTest_computer_security_5',
  'harness_hendrycksTest_conceptual_physics_5',
  'harness_hendrycksTest_econometrics_5',
  'harness_hendrycksTest_electrical_engineering_5',
  'harness_hendrycksTest_elementary_mathematics_5',
  'harness_hendrycksTest_formal_logic_5',
 

Loading leaderboard data:

In [3]:
with open('data/lb.pickle', 'rb') as handle:
    data = pickle.load(handle)

In this dataset, we have data from 395 models. Let's see the names of some of them below

In [4]:
len(data['models']),data['models'][:10]

(395,
 ['open-llm-leaderboard/details_zhengr__MixTAO-7Bx2-MoE-DPO',
  'open-llm-leaderboard/details_alignment-handbook__zephyr-7b-sft-full',
  'open-llm-leaderboard/details_rombodawg__Leaderboard-killer-MoE_4x7b',
  'open-llm-leaderboard/details_FelixChao__ExtremeDolphin-MoE',
  'open-llm-leaderboard/details_LoSboccacc__orthogonal-2x7B-base',
  'open-llm-leaderboard/details_moreh__MoMo-70B-lora-1.8.6-DPO',
  'open-llm-leaderboard/details_deepseek-ai__deepseek-moe-16b-base',
  'open-llm-leaderboard/details_Swisslex__Mixtral-Orca-v0.1',
  'open-llm-leaderboard/details_wang7776__Mistral-7B-Instruct-v0.2-sparsity-20',
  'open-llm-leaderboard/details_nfaheem__Marcoroni-7b-DPO-Merge'])

Below, we will process the data so all correctness scores (for all scenarios) are stored in $Y$. The dictionaries `scenarios_position` and `subscenarios_position` give the position of scenarios/subscenarios correctness scores in $Y$.

In [5]:
scenarios_position, subscenarios_position = prepare_data(scenarios, data)
Y = create_responses(scenarios, data)
Y.shape

(395, 28659)

For example, below you can see the scores for MMLU:

In [6]:
Y[:,scenarios_position['mmlu']], Y[:,scenarios_position['mmlu']].shape

(array([[0., 0., 1., ..., 1., 1., 0.],
        [0., 0., 1., ..., 1., 1., 0.],
        [0., 0., 1., ..., 1., 1., 0.],
        ...,
        [0., 0., 1., ..., 1., 1., 0.],
        [0., 0., 1., ..., 1., 1., 0.],
        [1., 0., 1., ..., 1., 1., 0.]]),
 (395, 14042))

For scenarios that have multiple subscenarios, it is usually the case that we want to give equal importance to individual subscenarios when computing the aggregated performance in that scenario. This is equivalent to using a weighted average when computing the aggregated performance. We will create balance_weights, a vector of weights to help us compute those weighted averages. These weights will be different than one only for MMLU, which is the only scenario with multiple subscenarios.

We will use this when choosing the IRT dimension.

In [7]:
balance_weights = np.ones(Y.shape[1])

N = len(scenarios_position['mmlu'])
n_sub = len(scenarios['mmlu'])
for sub in scenarios['mmlu']:
    n_i = len(subscenarios_position['mmlu'][sub])
    balance_weights[subscenarios_position['mmlu'][sub]] = N/(n_sub*n_i)  

We can see below that first averaging within subscenarios and then computing a simple average is equivalent to using a weighted average from the beginning:

In [8]:
accs1 = np.mean([Y[:,subscenarios_position['mmlu'][sub]].mean(axis=1) for sub in scenarios['mmlu']], axis=0)
accs2 = (balance_weights*Y)[:,scenarios_position['mmlu']].mean(axis=1)

np.abs(accs1 - accs2).mean()

2.322333605307685e-14

## Training IRT

Let's split the data in train and test (recent models are placed in the test set):

In [9]:
Y_test = Y[:100]
Y_train = Y[100:]

To train the IRT model, we first need to binarize the values in $Y$ because correctness is not binary for TruthfulQA.

In [10]:
Y_bin_train = np.zeros(Y_train.shape)
Y_bin_test = np.zeros(Y_test.shape)

cs = np.linspace(0.01,.99,100)  # Threshold values to consider
for scenario in scenarios.keys():
    ind = scenarios_position[scenario]
    # Find the best threshold value that minimizes the difference between averages
    c = cs[np.argmin([np.mean((np.abs((Y_train[:,ind]>c).mean(axis=1)-Y_train[:,ind].mean(axis=1)))) for c in tqdm(cs)])]
    # Apply the threshold to train and test responses
    Y_bin_train[:,ind] = (Y_train[:,ind]>c).astype(int)
    Y_bin_test[:,ind] = (Y_test[:,ind]>c).astype(int)

100%|█████████████████████████████████████████████| 100/100 [00:04<00:00, 24.05it/s]


Choosing the dimension for the IRT model based on a simple validation heuristic:

In [11]:
Ds = [5,10] # Dimensions to try
device = 'cuda' # Either 'cuda' or 'cpu' 
epochs = 2000  # Number of epochs for IRT model training (py-irt default is 2000)
lr = .1  # Learning rate for IRT model training (py-irt default is .1)

val_ind = list(range(0,Y_bin_train.shape[0],5)) # Validation indices
train_ind = [i for i in range(Y_bin_train.shape[0]) if i not in val_ind]

# Saving the training dataset in the needed format
create_irt_dataset(Y_bin_train[train_ind], 'data/irt_val_dataset.jsonlines')

# Trying different Ds
errors = []  
errors2 = []

for D in tqdm(Ds):
    dataset_name = 'data/irt_val_dataset.jsonlines'
    model_name = 'data/irt_val_model/'
    
    # Load trained IRT model parameters
    train_irt_model(dataset_name, model_name, D, lr, epochs, device)
    A, B, Theta = load_irt_parameters(model_name)
    
    # Determine seen and unseen items for validation
    seen_items = list(range(0, Y_bin_train.shape[1], 2))
    unseen_items = list(range(1, Y_bin_train.shape[1], 2))

    # Estimate ability parameters for the validation set
    thetas = [estimate_ability_parameters(Y_bin_train[val_ind][j][seen_items], A[:, :, seen_items], B[:, :, seen_items]) for j in range(len(val_ind))]

    # Compute validation errors for each scenario and update the errors list (in the end, we give the same weight for all scenarios)
    errors2.append([])
    for scenario in scenarios.keys():
        ind = [u for u in unseen_items if u in scenarios_position[scenario]]
        errors2[-1].append(np.mean([abs((balance_weights*item_curve(thetas[j], A, B))[0,ind].mean()-Y_train[val_ind][j,ind].mean())for j in range(len(val_ind))]))
    errors.append(np.mean(errors2[-1]))

  0%|                                                         | 0/2 [00:00<?, ?it/s]

[20:56:16] config: model_type='multidim_2pl' epochs=2000              cli.py:109
           priors='hierarchical' initializers=[] dims=5 lr=0.1                  
           lr_decay=0.9999 dropout=0.5 hidden=100 vocab_size=None               
           log_every=200 seed=42 deterministic=True                             
           data_path: data/irt_val_dataset.jsonlines                  cli.py:111
           output directory: data/irt_val_model/                      cli.py:112
[20:56:19] amortized: False                                       dataset.py:112
[20:56:38] Vocab size: None                                       training.py:90
[20:56:40] Training Model...                                          cli.py:116
[20:56:40] args: {'device': 'cuda', 'num_items': 28659,          training.py:134
           'num_subjects': 240}                                                 
           Parsed Model Args: {'device': 'cuda', 'num_items':    training.py:147
           28659, 'num_subje

 50%|████████████████████████                        | 1/2 [03:29<03:29, 209.63s/it]

[20:59:47] config: model_type='multidim_2pl' epochs=2000              cli.py:109
           priors='hierarchical' initializers=[] dims=10 lr=0.1                 
           lr_decay=0.9999 dropout=0.5 hidden=100 vocab_size=None               
           log_every=200 seed=42 deterministic=True                             
           data_path: data/irt_val_dataset.jsonlines                  cli.py:111
           output directory: data/irt_val_model/                      cli.py:112
[20:59:49] amortized: False                                       dataset.py:112
[21:00:09] Vocab size: None                                       training.py:90
[21:00:10] Training Model...                                          cli.py:116
[21:00:10] args: {'device': 'cuda', 'num_items': 28659,          training.py:134
           'num_subjects': 240}                                                 
           Parsed Model Args: {'device': 'cuda', 'num_items':    training.py:147
           28659, 'num_subje

100%|████████████████████████████████████████████████| 2/2 [07:32<00:00, 226.08s/it]


In [12]:
ind_D = np.argmin(np.array(errors))
D = Ds[ind_D]

Saving the training dataset in the needed format:

In [13]:
create_irt_dataset(Y_bin_train, 'data/irt_dataset.jsonlines')

To train the IRT model, we use an adapted version of `py-irt` code (please check README in the tutorials directory for mode details).

In [14]:
train_irt_model(dataset_name='data/irt_dataset.jsonlines', 
                model_name='data/irt_model', 
                D=D, lr=lr, epochs=epochs, device=device)               

[21:03:56] config: model_type='multidim_2pl' epochs=2000              cli.py:109
           priors='hierarchical' initializers=[] dims=10 lr=0.1                 
           lr_decay=0.9999 dropout=0.5 hidden=100 vocab_size=None               
           log_every=200 seed=42 deterministic=True                             
           data_path: data/irt_dataset.jsonlines                      cli.py:111
           output directory: data/irt_model                           cli.py:112
[21:04:00] amortized: False                                       dataset.py:112
[21:04:24] Vocab size: None                                       training.py:90
[21:04:25] Training Model...                                          cli.py:116
[21:04:25] args: {'device': 'cuda', 'num_items': 28659,          training.py:134
           'num_subjects': 300}                                                 
           Parsed Model Args: {'device': 'cuda', 'num_items':    training.py:147
           28659, 'num_subje

## Get the values for $\lambda$ which will be used to get the gp-IRT estimates (in conjunction with anchor methods)

In [15]:
def get_lambda(b, v):
    return (b**2)/(v+(b**2))

The variable `number_item` gives the number of data points we will sample per scenario:

In [16]:
number_item = 100

In [17]:
lambds = {} 

for i,scenario in enumerate(scenarios.keys()):
    v = np.var(Y_train[:,scenarios_position[scenario]], axis=1).mean()
    b = np.mean(errors2[ind_D][i]) 
    lambds[scenario] = get_lambda(b, v/(4*number_item))

In [18]:
with open('data/lambds.pickle', 'wb') as handle:
    pickle.dump(lambds, handle, protocol=pickle.HIGHEST_PROTOCOL)